***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [2]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-1e8d  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [3]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [4]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec 30 10:07 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [7]:
spark

In [18]:
# Put your bucket name below and make sure you can access it without an error
# bucket_name = 'ir_assg3_eithan' 
# full_path = f"gs://{bucket_name}/"
# paths=[]

# client = storage.Client()
# blobs = client.list_blobs(bucket_name)
# for b in blobs:
#     if b.name != 'graphframes.sh' and b.name !='small_df.parquet':
#         paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [19]:
# import json
# with open('/home/dataproc/queries_train.json', 'rt') as f:
#   queries = json.load(f)
# doc_set = []
# for docs in queries.values():
#   doc_set += docs
# docs_set = set(doc_set)
# print(len(docs_set))

1189


In [8]:
# parquetFile = spark.read.parquet(*paths)
# doc_text_pairs = parquetFile.select("text", "id").rdd

# small_rdd = parquetFile.rdd.filter(lambda row: row["id"] in docs_set)

NameError: name 'paths' is not defined

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [18]:
# Count number of wiki pages
# parquetFile.count()

In [22]:
# small_rdd = parquetFile.rdd.filter(lambda row: row["id"] in docs_set)
# small_rdd.count()

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [17]:
# small_df = small_rdd.toDF()
# small_df.show()

In [ ]:
# small_df.write.parquet(f'gs://{bucket_name}/small_df.parquet')

In [9]:
bucket_name = 'ir_assg3_eithan'
full_path2 = f"gs://{bucket_name}/"
paths_small=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name, prefix='small_df.parquet/part')
for b in blobs:
    paths_small.append(full_path2+b.name)
our_parquet = spark.read.parquet(*paths_small)

In [10]:
our_parquet.count()

1189

In [11]:
# our_rdd = our_parquet.rdd.filter(lambda row: row["id"]==844)
# our_rdd.toDF().show()
doc_text_pairs = our_parquet.select("text", "id").rdd


In [12]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [13]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [14]:
from inverted_index_gcp import InvertedIndex

<H1> Creating a small Index </H1>

<H1> Helper function to create Indexes <H1/>

In [15]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)


NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    cnt = Counter([tok for tok in tokens if tok not in all_stopwords])
    return [(key, (id, val)) for key,val in cnt.items()]
    
    
def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl, key= lambda tup: tup[0])


def calculate_df(postings):
    return postings.map(lambda lm: (lm[0],len(lm[1])))


def partition_postings_and_write(postings):
    partition_rdd = postings.map(lambda w_pl: (token2bucket_id(w_pl[0]),(w_pl[0],w_pl[1]))).groupByKey()
    return partition_rdd.map(lambda b_w_pl: InvertedIndex.write_a_posting_list((b_w_pl[0], b_w_pl[1]),bucket_name))


def word_count_binary(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    words_set = set(tok for tok in tokens if tok not in all_stopwords)
    return [(word, (id, 1)) for word in words_set]

# Assignment2 Code: 

def tokenize(text):
  return [token.group() for token in RE_WORD.finditer(text.lower())]

def calculate_tf(docs):
  word_counts = Counter()
  for text in docs:
    tokens = tokenize(title)
    doc_counts = Counter(tokens)
    word_counts.update(doc_counts)
  return word_counts

<H1>Creating Body Small Index<H1/>

In [16]:
# word counts map
word_counts_body = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_body = word_counts_body.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
#----------------------------------------------On big corpus change back to 50 --------------------------------------
postings_filtered = postings_body.filter(lambda x: len(x[1])>10)
w2df_body = calculate_df(postings_filtered)
w2df_body_dict = w2df_body.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()

# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    
# Create inverted index instance
body_index = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
body_index.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
body_index.df = w2df_body_dict
# write the global stats out
body_index.write_index('.', 'body_index')
# upload to gs
index_src = "body_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/small_indexes/{index_src}'
!gsutil cp $index_src $index_dst

!gsutil ls -lh $index_dst

Copying file://body_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][469.1 KiB/469.1 KiB]                                                
Operation completed over 1 objects/469.1 KiB.                                    
469.07 KiB  2022-12-30T16:06:11Z  gs://ir_assg3_eithan/postings_gcp/small_indexes/body_index.pkl
TOTAL: 1 objects, 480327 bytes (469.07 KiB)


In [96]:
# from inverted_index_gcp import MultiFileReader
# file_reader = MultiFileReader()
# print(file_reader.read(body_index.posting_locs[0], 1428))
# print(postings_filtered.take(1))

[('populous', [(844, 2), (1998, 4), (2377, 1), (3138, 4), (3708, 1), (5407, 15), (8522, 14), (10992, 2), (12521, 2), (14533, 4), (14597, 1), (14598, 2), (15474, 1), (18402, 2), (19058, 1), (19189, 4), (19261, 1), (22309, 2), (22989, 2), (27862, 2), (30057, 3), (31453, 3), (32706, 3), (34361, 1), (36511, 1), (37534, 4), (45470, 4), (47905, 3), (49728, 4), (53446, 7), (53837, 3), (69336, 1), (72214, 1), (124779, 1), (172538, 1), (231623, 4), (302201, 1), (309890, 10), (311130, 1), (314356, 4), (320443, 1), (390875, 7), (453584, 1), (636806, 6), (645042, 8), (1649321, 3), (3799826, 2), (5201333, 1), (6622547, 2), (7564733, 1), (11164587, 1), (15218891, 1), (19159283, 1), (20587357, 1)])]


<H1> Creating Title Small Index <H1/>

In [17]:
doc_title_pairs = our_parquet.select("title", "id").rdd


# word counts map
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count_binary(x[0], x[1]))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
#----------------------------------------------On big corpus change back to 50 --------------------------------------
# postings_filtered = postings_title.filter(lambda x: len(x[1])>10)
w2df_title = calculate_df(postings_title)
w2df_title_dict = w2df_title.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()

# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    
# Create inverted index instance
title_index = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
title_index.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
title_index.df = w2df_title_dict
# write the global stats out
title_index.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/small_indexes/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][324.6 KiB/324.6 KiB]                                                
Operation completed over 1 objects/324.6 KiB.                                    


In [106]:
# print(title_index.df)

{'inc': 11, 'series': 12, 'disorder': 13, 'wine': 33, 'world': 60, 'cup': 55, 'gold': 25, 'java': 40, 'language': 15, 'money': 22, 'netflix': 14, 'marvel': 26, 'united': 12, 'universe': 19, 'rick': 15, 'apple': 21, 'fifa': 47, 'india': 35, 'information': 24, 'retrieval': 28, 'list': 66, 'jordan': 16, 'simpsons': 27, 'winter': 41, 'cannabis': 37, 'cigarette': 36, 'children': 12, 'depression': 26, 'cities': 13, 'coffee': 34, 'weight': 12, 'film': 14, 'states': 11, 'season': 19, 'qualification': 12, '2022': 21, 'morty': 15, 'cinematic': 18}


<H1> Creating Anchor Text Small Inverted Index <H1/>

In [22]:
doc_anchor_pairs = our_parquet.select("anchor_text", "id").rdd

def parse_anchor_text(pair):
    id = pair[1]
    anchor_text = ""
    for tup in pair[0]:
        anchor_text += " " + tup[1]
    return (anchor_text, id)

tmp_pairs = doc_anchor_pairs.map(lambda pair: parse_anchor_text(pair))
# print(tmp_pairs.take(1))

In [23]:

    
# word counts map
word_counts_anchor = tmp_pairs.flatMap(lambda x: word_count_binary(x[0], x[1]))
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)
w2df_anchor = calculate_df(postings_anchor)
w2df_anchor_dict = w2df_anchor.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()

# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    
# Create inverted index instance
anchor_index = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
anchor_index.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
anchor_index.df = w2df_anchor_dict
# write the global stats out
anchor_index.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/small_indexes/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://anchor_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      


In [24]:
# print(list(anchor_index.df.keys())[:100])
# print(len(anchor_index.df.items()))
# print(len(title_index.df.items()))
# print(len(body_index.df.items()))

['camps', 'illegal', 'gerrit', 'planological', 'laan', 'refugee', 'carsharing', 'intercity-express', 'community', 'huguenot', 'count', 'toon', 'keyser', 'eberhard', 'afc', 'vingboons', 'work', 'university', 'netflix', 'mart', 'world', 'uitmarkt', 'schiphol', 'arkefly', 'attractions', 'hendrick', 'new', 'diemen', 'hermitage', 'effect', 'defined', 'school', 'climate', 'vervoerbedrijf', 'sugar', 'zaanstad', 'islam', 'hortus', 'indonesia', 'america', 'uber', 'forbes', 'james', 'revolution', 'molenaar', 'nelis', 'thirty', 'fontainebleau', 'sbs', 'historic', 'social', 'modern', 'lyceum', 'canals', 'bibliotheek', 'toneelgroep', 'inquisition', 'palace', 'bergen-belsen', 'applied', 'magistra', 'raw', 'twelve', 'low', 'asylum', 'austria', 'martinair', 'countries', 'amstelpark', 'vincent', 'green', 'paths', 'scotland', 'rent', 'ensign', 'samuel', 'pearl', 'area', 'east', 'stadium', 'flevopark', 'revolt', 'michel', 'lycée', 'guest', 'sea', 'concentration', 'reserve', "king's", 'tax', 'king', 'muse

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [33]:
# Put your `generate_graph` function here
def generate_graph(pages):
    distinct_rdd =  pages.map(lambda page: (page[0], set(map(lambda id_text: id_text[0], page[1]))))
    edges = distinct_rdd.flatMap(lambda src_page: list(map(lambda dst_page: (src_page[0], dst_page),src_page[1])))
    vertices = edges.flatMap(lambda edge: [(edge[0],),(edge[1],)]).distinct()
    return edges, vertices

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750| 9913.728782160773|
|  10568| 5385.349263642039|
|  32927| 5282.081575765276|
|  30680| 5128.233709604121|
|5843419| 4957.567686263867|
|  68253| 4769.278265355161|
|  31717|  4486.35018054831|
|  11867|4146.4146509127695|
|  14533| 3996.466440885503|
| 645042| 3531.627089803743|
|  17867|3246.0983906041415|
|5042916|2991.9457391661786|
|4689264| 2982.324883041747|
|  14532|2934.7468292031717|
|  25391| 2903.546223513398|
|   5405|2891.4163291546356|
|4764461| 2834.366987332661|
|  15573| 2783.865118158838|
|   9316| 2782.039646413769|
|8569916|2775.2861918400163|
+-------+------------------+
only showing top 20 rows

